In [1]:
from bs4 import BeautifulSoup as bs
import urllib2
import requests
import re
import datetime as dt
import csv
from apscheduler.schedulers.blocking import BlockingScheduler

In [2]:
def csv_writer(data, path):
  
        with open(path, "a") as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(data)

In [3]:
def dupe_delete(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [24]:
def view_extractor(url,title):
    
    #url="https://www.youtube.com/watch?v=1nCLBTmjJBY"
    page = urllib2.urlopen(url)
    data = page.read()
    soup = bs(data, 'html.parser')
    
    s = soup.find_all("script")   #searching by script, 
    scripts = []

    for x in s:
        scripts.append(str(x))   #apparently view count is in a script. String because it's all alphanumeric crap
    
    views = []
    
    for line in scripts:
        m = re.search('view_count(.+?),', line)   #this was picked up from stackexchange, very handy
        if m:
            found = m.group(1)
            views.append(found)
    vcount = views[0]
    vstr = vcount.strip('\":\'').split()
    count = int(vstr[0])  #convert the string into an integer
    
    #timestamp
    d = dt.datetime.now()    
    timestamp = dt.datetime.strftime(d, '%H:%M:%S')
    datestamp = dt.datetime.strftime(d, '%m/%d/%y')

    #Views, Date, Time are distinct columns
    dater = (count,datestamp,timestamp)
    data = list(dater)
    
    

    csv_writer(data, title)
    print('Iteration complete:',datestamp,timestamp)

In [8]:
def searcher():
    x = raw_input("Enter keyword:")
    l = str(x)
    ur = 'https://www.youtube.com/results?search_query='
    url = ur+l
    
    r = requests.get(url)
    data = r.text
    
    links = []
    soup = bs(data, 'lxml')
    for link in soup.find_all('a'):
        listy = link.get('href')
        links.append(listy)
    
    linklist= []
    for element in links:
        item = [element for element in links if '/watch?v=' in element]
        linklist = item
        
    uniques = dupe_delete(linklist)
    return uniques

In [16]:
videos = searcher()

Enter keyword:4minute


In [37]:
url = 'https://www.youtube.com/watch?v=1nCLBTmjJBY'

In [42]:
view_extractor(url,'test124')

('Iteration complete:', '11/24/17', '13:56:53')
